<a href="https://colab.research.google.com/github/mkrsteska/BSA2020_Team_Tissot_Project_2/blob/master/code/BERT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT

In [0]:
!pip install torch


In [0]:
!pip install pytorch-pretrained-bert pytorch-nlp -q
!pip install keras -q

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/mkrsteska/BSA2020_Team_Tissot_Project_2/master/data/train.csv")

In [0]:
df.dtypes

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

In [0]:
df.keyword = df.keyword.astype("str")
df.text = df.text.astype("str")
df.location = df.location.astype("str")

In [0]:
#@title Preproccesing
import re
import sys
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

import string


GLOVE_DIMENSION = 25
MAX_WORDS = 30 

CONTRACTIONS = { 
    "ain't": "am not",
    "aren't": "are not",
    "amn't": "am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "daren't": "dare not",
    "daresn't": "dare not",
    "dasn't": "dare not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "e'er": "ever",
    "everyone's": "everyone is",
    "finna": "going to",
    "gimme": "give me",
    "giv'n": "given",
    "gonna": "going to",
    "gon't": "go not",
    "gotta": "got to",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",    
    "he've": "he have",
    "howdy": "how do you do",
    "how're": "how are",
    "i'd": "I had",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "i'm'a": "I am about to",
    "i'm'o": "I am going to",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "ne'er": "never",
    "o'clock": "of the clock",
    "o'er": "over",
    "ol'": "old",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "somebody's": "somebody is",
    "someone's": "someone is",
    "something's": "something is",
    "so've": "so have",
    "so's": "so is",
    "so're": "so are", 
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they had",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "'tis": "it is",
    "'twas": "it was",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

ABBREVIATIONS = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

'''Function to preprocess the tweets'''
def preprocess_tweet(tweet):
    # MULTILINE - '^' matches the beggining of each line
    # DOTALL - '.' matches every character including newline
    FLAGS = re.MULTILINE | re.DOTALL
          
    # Replace links with token <url>
    tweet = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ", tweet, flags = FLAGS)
    
    # Remove hashtags
    tweet = re.sub(r"\#","", tweet, flags = FLAGS)
    
    # Remove mentions, starting with @
    tweet = re.sub(r'@\w+', '', tweet, flags = FLAGS)
            
    # Eyes of a smiley can be represented with: 8:=;
    # Nose of a smiley can be represented with: '`\-
    
    # Replace smiling face with <smile>. Mouth can be repredented with: )dD.
    tweet = re.sub(r"[8:=;]['`\-]?[)dD]+|[(dD]+['`\-]?[8:=;]", " <smile> ", tweet, flags = FLAGS)
    
    # Replace lol face with <lolface>. Mouth can be represented with: pP
    tweet = re.sub(r"[8:=;]['`\-]?[pP]+", " <lolface> ", tweet, flags = FLAGS)
    
    # Replace sad face with <sadface>. Mouth can be represented with: (
    tweet = re.sub(r"[8:=;]['`\-]?[(]+|[)]+['`\-]?[8:=;]", " <sadface> ", tweet, flags = FLAGS)
    
    # Replace neutral face with <neutralface>. Mouth can be represented with: \/|l
    tweet = re.sub(r"[8:=;]['`\-]?[\/|l]+", " <neutralface>", tweet, flags = FLAGS)
    
    # Split concatenated words wih /. Ex. Good/Bad -> Good Bad
    tweet = re.sub(r"/"," / ", tweet, flags = FLAGS)
    
    # Replace <3 with <heart>
    tweet = re.sub(r"<3"," <heart> ", tweet, flags = FLAGS)
    
    # Replace numbers with <number>.
    tweet = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ", tweet, flags = FLAGS)
    
    # Replace repeated punctuation with <repeat>. Ex. !!! -> ! <repeat>
    tweet = re.sub(r"([!?.]){2,}", r"\1 <repeat> ", tweet, flags = FLAGS)
    
     # Replace elongated endings with <elong>. Ex. happyyy -> happy <elong>
    tweet = re.sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong> ", tweet, flags = FLAGS)
       
    # Expand English contractions
    for word in tweet.split():
        if word.lower() in CONTRACTIONS:
            tweet = tweet.replace(word, CONTRACTIONS[word.lower()])
            
    # Expand abbreviations
    for word in tweet.split():
        if word.lower() in ABBREVIATIONS:
            tweet = tweet.replace(word, ABBREVIATIONS[word.lower()])
            
    # Remove apostrophes
    tweet = re.sub(r"'","", tweet, flags = FLAGS)

    # Add space between punctuation and word
    tweet = re.sub(r'(?<=[^ ])(?=[.,!?()])|(?<=[.,!?()])(?=[^ ])', r' ', tweet, flags = FLAGS)
    
    # Replace multiple empty spaces with one
    tweet = re.sub('\s+', ' ', tweet, flags = FLAGS)
       
    # Convert all tokens to lowercase
    tweet = tweet.lower()
    
    # Return result
    return tweet

def generate_embedding_matrix():
    df_train = pd.read_csv("../data/train.csv")
    df_test = pd.read_csv("../data/test.csv")
    
    train_tweets = []  
    test_tweets = []  
    labels = []
    
    # Preprocessing the train tweets
    for row in df_train.iterrows():
        tweet = preprocess_tweet(row[1]['text'])
        train_tweets.append(tweet)
        labels.append(row[1]['target'])
        
    y_train = labels
    
    # Preprocessing the test tweets
    for row in df_test.iterrows():
        tweet = preprocess_tweet(row[1]['text'])
        test_tweets.append(tweet)
        
    # Mapping every unique word to a integer (bulding the vocabulary)
    word_to_index = {}
    words_freq = {}
    m = 0
    
    for i, tweet in enumerate(train_tweets):
        words = tweet.split()
        
        for word in words[:MAX_WORDS]:
            if word not in word_to_index:
                word_to_index[word] = m
                m += 1
            if word not in words_freq:
                words_freq[word] = 1
            else:
                words_freq[word] += 1
            
    word_to_index["unk"] = m
    vocabulary_size = len(word_to_index)
            
    # Converting training tweets to integer sequences
    train_sequences = []

    for i, tweet in enumerate(train_tweets):
        words = tweet.split()

        tweet_seq = []
        for word in words[:MAX_WORDS]:
            if word not in word_to_index:
                tweet_seq.append(word_to_index["unk"])
            else:
                tweet_seq.append(word_to_index[word])

        train_sequences.append(tweet_seq)
    
    # Padding the sequences to match the `MAX_WORDS`
    X_train = pad_sequences(train_sequences, maxlen=MAX_WORDS, padding="post", value=vocabulary_size)
    
    # Converting test tweets to integer sequences
    test_sequences = []

    for i, tweet in enumerate(test_tweets):
        words = tweet.split()

        tweet_seq = []
        for word in words[:MAX_WORDS]:
            if word not in word_to_index:
                tweet_seq.append(word_to_index["unk"])
            else:
                tweet_seq.append(word_to_index[word])

        test_sequences.append(tweet_seq)
    
    # Padding the sequences to match the `MAX_WORDS`
    X_test = pad_sequences(test_sequences, maxlen=MAX_WORDS, padding="post", value=vocabulary_size)
    
   # Reading glove embeddings
    glove_embeddings_file = open('../data/glove.twitter.27B.25d.txt', 'r', encoding='UTF-8')

    glove_embeddings = dict()
    for line in glove_embeddings_file:
        parts = line.split()
        key = parts[0]
        embedding = [float(t) for t in parts[1:]]
        glove_embeddings[key] = np.array(embedding)

    # Generating the embedding matrix for our vocabulary (this is needed for the Embedding layer in keras models)
    unknown = []
    hits = 0
    embedding_matrix = np.zeros((vocabulary_size + 1, GLOVE_DIMENSION))
    for word, idx in word_to_index.items():
        if word in glove_embeddings:
            emb = glove_embeddings[word]
            embedding_matrix[idx] = emb
            hits += 1
    else:
        unknown.append(word)
        emb = glove_embeddings["unk"]
        embedding_matrix[idx] = emb
    
    embedding_matrix[vocabulary_size] = [0]*GLOVE_DIMENSION
        
    return (X_train, y_train, X_test, embedding_matrix)


'''Function to remove English stopwords from a Pandas Series.'''
def remove_stopwords(input_text):
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 



'''Function to preprocess the tweets'''
def preprocess_tweet_use(tweet):
    # MULTILINE - '^' matches the beggining of each line
    # DOTALL - '.' matches every character including newline
    FLAGS = re.MULTILINE | re.DOTALL
          
    # Replace links
    tweet = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " ", tweet, flags = FLAGS)
    
    # Remove hashtags
    tweet = re.sub(r"\#","", tweet, flags = FLAGS)
    
    # Remove mentions, starting with @
    tweet = re.sub(r'@\w+', '', tweet, flags = FLAGS)
            
    # Eyes of a smiley can be represented with: 8:=;
    # Nose of a smiley can be represented with: '`\-
    
    # Replace smiling face with <smile>. Mouth can be repredented with: )dD.
    tweet = re.sub(r"[8:=;]['`\-]?[)dD]+|[(dD]+['`\-]?[8:=;]", " smile ", tweet, flags = FLAGS)
    
    # Replace lol face with <lolface>. Mouth can be represented with: pP
    tweet = re.sub(r"[8:=;]['`\-]?[pP]+", " lol ", tweet, flags = FLAGS)
    
    # Replace sad face with <sadface>. Mouth can be represented with: (
    tweet = re.sub(r"[8:=;]['`\-]?[(]+|[)]+['`\-]?[8:=;]", " sad ", tweet, flags = FLAGS)
    
    # Replace neutral face with <neutralface>. Mouth can be represented with: \/|l
    tweet = re.sub(r"[8:=;]['`\-]?[\/|l]+", " neutral", tweet, flags = FLAGS)
    
    # Split concatenated words wih /. Ex. Good/Bad -> Good Bad
    tweet = re.sub(r"/"," / ", tweet, flags = FLAGS)
    
    # Replace <3 with <heart>
    tweet = re.sub(r"<3"," heart ", tweet, flags = FLAGS)
    
    # Replace elongated endings with <elong>. Ex. happyyy -> happy <elong>
    tweet = re.sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong> ", tweet, flags = FLAGS)
    
    # Remove <elong>
    tweet = re.sub(r"<elong>", r" ", tweet, flags = FLAGS)
    
    # Expand English contractions
    for word in tweet.split():
        if word.lower() in CONTRACTIONS:
            tweet = tweet.replace(word, CONTRACTIONS[word.lower()])
            
    # Expand abbreviations
    for word in tweet.split():
        if word.lower() in ABBREVIATIONS:
            tweet = tweet.replace(word, ABBREVIATIONS[word.lower()])

    # Remove punctuation
    tweet = tweet.strip(string.punctuation)
    
    # Replace multiple empty spaces with one
    tweet = re.sub('\s+', " ", tweet, flags = FLAGS)
       
    # Convert all tokens to lowercase
    tweet = tweet.lower()
    
    # Return result
    return tweet

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
df.text = df.text.apply(preprocess_tweet)

In [0]:
# Create sentence and label lists
sentences = df.text.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.target.values

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['[CLS]', 'our', 'deeds', 'are', 'the', 'reason', 'of', 'this', 'earthquake', 'may', 'allah', 'forgive', 'us', 'all', '[SEP]']


In [0]:
df.text.str.len().max()

211

In [0]:
# Set the maximum length of a tweet is 280. The longest sequence in our training set is 157, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 400

In [0]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [0]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Create the attention masks

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [0]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=2)

In [0]:
train_labels = train_labels.astype("int")
validation_labels = validation_labels.astype("int")

In [0]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [0]:
#In order for torch to use the GPU, we need to identify and specify the GPU as the device. Later, in our training loop, we will load data onto the device. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
t = [] 

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    
    loss = model(b_input_ids.to(torch.int64), token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CP
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:  25%|██▌       | 1/4 [06:37<19:51, 397.08s/it]

Train loss: 0.4739609623030454
Validation Accuracy: 1.0


Epoch:  50%|█████     | 2/4 [13:13<13:14, 397.01s/it]

Train loss: 0.3217339473737388
Validation Accuracy: 1.0


Epoch:  75%|███████▌  | 3/4 [19:51<06:37, 397.08s/it]

Train loss: 0.21517997930812485
Validation Accuracy: 1.0


Epoch: 100%|██████████| 4/4 [26:28<00:00, 397.01s/it]

Train loss: 0.12729078815357775
Validation Accuracy: 1.0


Let's try on the test data set.

We have to prepare it similarly to the train data

In [0]:
df_test = pd.read_csv("https://raw.githubusercontent.com/mkrsteska/BSA2020_Team_Tissot_Project_2/master/data/test.csv")

df_test.text = df_test.text.apply(preprocess_tweet)

# Create sentence and label lists
sentences = df_test.text.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
#labels = df_test["target"]

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 300

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [0]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions = []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Move logits to CPU
  logits = logits.detach().cpu().numpy()
  
  # Store predictions
  predictions.append(logits)

In [0]:
predictions[0]

array([[-2.0982003e+00,  2.7075412e+00],
       [-1.4971666e+00,  1.8905275e+00],
       [-2.3932636e+00,  2.9642613e+00],
       [-2.5514207e+00,  3.0971081e+00],
       [-2.8907380e+00,  3.6038306e+00],
       [-1.8889656e+00,  2.3833919e+00],
       [ 4.2097559e+00, -5.2078810e+00],
       [ 4.1179094e+00, -5.2066011e+00],
       [ 4.0940380e+00, -5.1609330e+00],
       [ 3.2732484e+00, -3.6682661e+00],
       [ 3.1310041e+00, -3.5203192e+00],
       [ 3.5028579e+00, -4.0386829e+00],
       [ 3.4424841e+00, -3.9560125e+00],
       [ 3.5840313e+00, -4.2106266e+00],
       [ 3.8160012e+00, -4.7084503e+00],
       [-2.4883497e+00,  3.0556436e+00],
       [ 4.1063776e+00, -5.0607071e+00],
       [-1.6959136e+00,  2.0992064e+00],
       [ 2.5599773e+00, -2.6790590e+00],
       [ 4.1203537e+00, -5.0874891e+00],
       [ 5.6385732e-01, -3.5042599e-01],
       [-4.7866660e-03,  2.9086500e-01],
       [ 3.8858845e+00, -4.7247858e+00],
       [-2.4642508e+00,  3.0305932e+00],
       [ 3.27888

In [0]:
# Flatten the predictions on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [0]:
flat_predictions

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
df_test["target"] = flat_predictions

In [0]:
df_test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,just happened a terrible car crash,1
1,2,NaN,NaN,"heard about earthquake is different cities , s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond , geese ar...",1
3,9,NaN,NaN,apocalypse lighting . spokane wildfires,1
4,11,NaN,NaN,typhoon soudelor kills <number> in china and t...,1


In [0]:
submission = df_test.drop(["keyword", "location", "text"], axis = 1)

In [0]:
submission_csv = submission.to_csv("11. Submission_BERT.csv", index = False, header=True)

In [0]:
from google.colab import files
#files.download('11. Submission_BERT.csv')

In [0]:
df_test_labeled = pd.read_csv("test_labeled.csv")

In [0]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(df_test["target"], df_test_labeled.target,digits= 4))

              precision    recall  f1-score   support

           0     0.8705    0.8270    0.8482      1959
           1     0.7582    0.8152    0.7857      1304

    accuracy                         0.8222      3263
   macro avg     0.8144    0.8211    0.8169      3263
weighted avg     0.8256    0.8222    0.8232      3263

